# Aula 3

Importando os pacotes necessários:

In [ ]:
import seaborn as sns
import networkx as nx

import freeman as fm

Função para carregar e dicotomizar uma rede de países:

In [ ]:
# Valores possíveis do parâmetro topic:
# ------------------------------------
# 'all': todos os tópicos
# 'co': colonialismo
# 'di': controle de armas e desarmamento
# 'ec': desenvolvimento econômico
# 'hr': Direitos Humanos
# 'me': Oriente Médio
# 'nu': armas nucleares e material nuclear


# Valores possíveis do parâmetro period:
# -------------------------------------
# '1946' a '2015': votações de um ano específico
# '1940s' a '2010s': votações de uma década específica
# 'all': todas as votações de 1946 a 2015


def load(topic, period):
    g = fm.load('un/{}/{}.gml'.format(topic, period))

    # Remover todas as arestas com peso menor ou igual a 0.5.
    # Precisamos de dois loops, pois não é uma boa ideia
    # tirar algo de um conjunto enquanto iteramos nele.
    removed = []
    for n, m in g.edges:
        if g.edges[n, m]['weight'] <= 0.5:
            removed.append((n, m))
    for n, m in removed:
        g.remove_edge(n, m)

    # Remover todos os nós que ficaram isolados depois da
    # remoção das arestas, para melhorar a visualização.
    removed = []
    for n in g.nodes:
        if not g.degree(n):
            removed.append(n)
    for n in removed:
        g.remove_node(n)

    return g

Função para visualizar uma rede dicotomizada:

In [ ]:
def draw(g):
    # Pintar o Brasil de verde.
    g.nodes['BR']['color'] = (0, 255, 0)

    # Deixar os nós menores e fazer os nomes deles
    # aparecer apenas quando o mouse passa por cima.
    g.set_all_nodes(size=10, labpos='hover')

    # Deixar as arestas semi-transparentes para
    # a visualização ficar um pouco mais leve.
    g.set_all_edges(color=(0, 0, 0, 0.25))

    # Usar um algoritmo especial para posicionar
    # os nós de acordo com grupos.
    g.move('kamada_kawai')

    g.draw()

Função para calcular as centralidades do Brasil:

*(parece ineficiente calcular tudo só para pegar a do Brasil, mas para os algoritmos mais eficientes não faz muita diferença... o cálculo envolve a rede inteira de qualquer forma)*

In [ ]:
def brazil_centralities(g):
    dc = nx.degree_centrality(g)
    bc = nx.betweenness_centrality(g)
    cc = nx.closeness_centrality(g)
    ec = nx.eigenvector_centrality(g, max_iter=1000)
    return dc['BR'], bc['BR'], cc['BR'], ec['BR']

Visualizando a rede de cada década:

*(as visualizações são meio pesadonas...o notebook vai dar umas travadas mas assim que terminar de renderizar vai voltar ao normal... espero)*

In [ ]:
for year in range(1940, 2020, 10):
    period = '{}s'.format(year)
    print(period)
    g = load('all', period)
    draw(g)

Calculando as centralidades do Brasil ao longo das décadas:

In [ ]:
years = list(range(1940, 2020, 10))

dc = []
bc = []
cc = []
ec = []

for year in years:
    period = '{}s'.format(year)
    g = load('all', period)
    d, b, c, e = brazil_centralities(g)
    dc.append(d)
    bc.append(b)
    cc.append(c)
    ec.append(e)
    print(period)

Plotando a centralidade degree do Brasil ao longo das décadas:

In [ ]:
sns.lineplot(years, dc);

Plotando a centralidade betweenness do Brasil ao longo das décadas:

In [ ]:
sns.lineplot(years, bc);

Plotando a centralidade closeness do Brasil ao longo das décadas:

In [ ]:
sns.lineplot(years, cc);

Plotando a centralidade eigenvector do Brasil ao longo das décadas:

In [ ]:
sns.lineplot(years, ec);

EOF